In [11]:
from scipy import stats as st
import pandas as pd

In [12]:
csv_path = "../dataset/preprocessed.csv"

In [13]:
data = pd.read_csv(csv_path)
data.describe()

,Age,Number of Cigarettes Beedi per day,Number of years smoked,Height,Weight,Heart_Rate,BP_Systolic,BP_Diastolic,Exact Time of Symptom Onset,BASELINE SERUM CREATININE,...,Time_since_symptom_onset_to_1st_ECG_3 HOURS,Time_since_symptom_onset_to_1st_ECG_30 HOURS,Time_since_symptom_onset_to_1st_ECG_4 HOURS,Time_since_symptom_onset_to_1st_ECG_48 HOURS,Time_since_symptom_onset_to_1st_ECG_5 HOURS,Time_since_symptom_onset_to_1st_ECG_6 HOURS,Time_since_symptom_onset_to_1st_ECG_7 HOURS,Time_since_symptom_onset_to_1st_ECG_72 HOURS,Time_since_symptom_onset_to_1st_ECG_8 HOURS,Time_since_symptom_onset_to_1st_ECG_9 HOURS
count,1.630000e+03,1.630000e+03,1.630000e+03,1.630000e+03,1.630000e+03,1.630000e+03,1.630000e+03,1.630000e+03,1.630000e+03,1.630000e+03,...,1630.000000,1630.000000,1630.000000,1630.000000,1630.000000,1630.000000,1630.000000,1630.000000,1630.000000,1630.000000
mean,-2.915187e-16,-8.500358e-17,-6.102821e-17,1.063635e-15,-5.056623e-16,2.397537e-16,1.569297e-16,4.577116e-16,4.577116e-17,-5.274581e-16,...,0.004908,0.073006,0.017178,0.007975,0.028221,0.042945,0.057055,0.004908,0.308589,0.006748
std,1.000307e+00,1.000307e+00,1.000307e+00,1.000307e+00,1.000307e+00,1.000307e+00,1.000307e+00,1.000307e+00,1.000307e+00,1.000307e+00,...,0.069906,0.260226,0.129974,0.088976,0.165654,0.202795,0.232019,0.069906,0.462053,0.081897
min,-3.048569e+00,-7.118193e-01,-7.171164e-01,-4.368857e+00,-2.517302e+00,-2.699300e+00,-3.391341e+00,-4.068557e+00,-1.772097e+00,-2.189102e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-6.995569e-01,-7.118193e-01,-7.171164e-01,-6.095799e-01,-7.113489e-01,-5.007379e-01,-5.141819e-01,-5.000698e-01,-8.615311e-01,-5.823365e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.548281e-02,-7.118193e-01,-7.171164e-01,8.868921e-02,-4.599786e-02,-1.253736e-01,-1.031591e-01,2.136276e-01,-9.007903e-02,-1.232605e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,6.427359e-01,6.458515e-01,6.045703e-01,6.679607e-01,7.144033e-01,5.181079e-01,6.366818e-01,9.273250e-01,8.837211e-01,3.358155e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,3.495108e+00,6.076535e+00,3.247944e+00,2.648261e+00,4.516409e+00,5.719584e+00,4.007068e+00,4.495812e+00,1.857521e+00,1.112410e+01,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [14]:
num = ['Age', 'Number of Cigarettes Beedi per day', 'Number of years smoked', 'Height', 'Weight', 'Heart_Rate', 'BP_Systolic', 'BP_Diastolic',
        'Exact Time of Symptom Onset', 'BASELINE SERUM CREATININE', 'HAEMOGLOBIN', 'HBA1C', 'Total_cholesterol', 'LDL', 'HDL', 'Triglycerides',
        'APO B(g/l)', 'Liporotein A', 'Admission Time','BMI','Pack_Year'
      ]

In [15]:
for col in data.columns:
    data[col] = data[col].fillna(0)

In [16]:
target = "Follow Up Status"

In [17]:
labels = data[target].unique()
labels

array(['Alive', 'Deceased'], dtype=object)

In [18]:
test_stats = {}
p_vals = {}

alpha = 0.05

In [19]:
for col in data.columns:
    if(data[col].unique().shape[0]==1):
        continue

    if (col != target):
        
        counts = [data[data[target] == label][col] for label in labels]

        if col not in num:
            observed = pd.crosstab(index = data[col], columns = data[target])
            test_stat, p_value, _, _ = st.chi2_contingency(observed)
        else:
            test_stat, p_value = st.ttest_ind(a=counts[0],b=counts[1],equal_var=False)

        test_stats[col] = test_stat
        p_vals[col] = p_value

In [20]:
results = pd.DataFrame({'Feature' : list(p_vals.keys()),
                        'CV' : list(test_stats.values()),
                        'p value' : list(p_vals.values()),
                        'Significant' : ['S' if p_val < alpha else 'X' for p_val in p_vals.values()]
                        })

results_xl_path = "feature_tests.xlsx"

results.to_excel(results_xl_path, index = False)